In [5]:
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import deepxde as dde
from mpl_toolkits import mplot3d
import numpy as np
import sys
print(sys.executable)


# 用于绘图


c:\tools\Anaconda3\python.exe


In [1]:
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import deepxde as dde
from mpl_toolkits import mplot3d
import numpy as np
import sys
print(sys.executable)


# 用于绘图


print(sys.executable)
# 周期性边界条件
x_lower = -5*np.pi
x_upper = 5*np.pi
t_lower = -2
t_upper = 2

# 创建 2D 域（用于绘图和输入）
x = np.linspace(x_lower, x_upper, 512)
t = np.linspace(t_lower, t_upper, 402)
X, T = np.meshgrid(x, t)

# 整个域变平
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

# 空间和时间域/几何（对于 deepxde 模型）
space_domain = dde.geometry.Interval(x_lower, x_upper)
time_domain = dde.geometry.TimeDomain(t_lower, t_upper)
geomtime = dde.geometry.GeometryXTime(space_domain, time_domain)
# 损失的“物理信息”部分


def pde(x, y):
    """
    INPUTS:
        x: x[:,0] 是 x 坐标
           x[:,1] 是 t 坐标
        y: 网络输出，在这种情况下:
            y[:,0] 是 u(x,t) 实部
            y[:,1] 是 v(x,t) 虚部
    OUTPUT:
        标准形式的 pde，即必须为零的东西
    """

    u1 = y[:, 0:1]
    v1 = y[:, 1:2]
    u2 = y[:, 2:3]
    v2 = y[:, 3:4]

    # 在'jacobian'中，i 是输出分量，j 是输入分量
    u1_t = dde.grad.jacobian(y, x, i=0, j=1)
    v1_t = dde.grad.jacobian(y, x, i=1, j=1)
    u2_t = dde.grad.jacobian(y, x, i=2, j=1)
    v2_t = dde.grad.jacobian(y, x, i=3, j=1)

    u1_x = dde.grad.jacobian(y, x, i=0, j=0)
    v1_x = dde.grad.jacobian(y, x, i=1, j=0)
    u2_x = dde.grad.jacobian(y, x, i=2, j=0)
    v2_x = dde.grad.jacobian(y, x, i=3, j=0)

    # 在“hessian”中，i 和 j 都是输入分量。 （Hessian 原则上可以是 d^2y/dxdt、d^2y/d^2x 等）
    # 输出组件由“组件”选择
    u1_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    v1_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    u2_xx = dde.grad.hessian(y, x, component=2, i=0, j=0)
    v2_xx = dde.grad.hessian(y, x, component=3, i=0, j=0)

    # f_u = u_t + 0.5 * v_xx + (u ** 2 + v ** 2) * v
    # f_v = v_t - 0.5 * u_xx - (u ** 2 + v ** 2) * u

    r1 = 1
    r2 = 2
    r3 = -1
    r4 = 0
    beta = 1

    f_u1 = (
        v1_t + u1_xx
        + r1*u1*(u1**2 + v1**2) + r2*u1*(u2**2 + v2**2) + r3*(u1*(u2 **
                                                                  2 - v2**2) + 2*u2*v1*v2) + r4*(u2*(u1**2 - v1**2) + 2*u1*v1*v2)
    )

    f_v1 = (
        -u1_t + v1_xx
        + r1*v1*(u1**2 + v1**2) + r2*v1*(u2**2 + v2**2) - r3*(v1*(u2 **
                                                                  2 - v2**2) - 2*u1*u2*v2) - r4*(v2*(u1**2 - v1**2) - 2*u1*u2*v1)
    )

    f_u2 = (
        v2_t + beta*u2_xx
        + r1*u2*(u2**2 + v2**2) + r2*u2*(u1**2 + v1**2) + r3*(u2*(u1 **
                                                                  2 - v1**2) + 2*u1*v2*v1) + r4*(u1*(u2**2 - v2**2) + 2*u2*v2*v1)
    )

    f_v2 = (
        -u2_t + beta*v2_xx
        + r1*v2*(u2**2 + v2**2) + r2*v2*(u1**2 + v1**2) - r3*(v2*(u1 **
                                                                  2 - v1**2) - 2*u2*u1*v1) - r4*(v1*(u2**2 - v2**2) - 2*u2*u1*v2)
    )

    return [
        f_u1, f_v1, f_u2, f_v2,
        # f_u1x, f_u1t,
        # f_v1x, f_v1t,
        # f_u2x, f_u2t,
        # f_v2x, f_v2t
    ]
 # 边界条件和初始条件


# 周期性边界条件
bc_u1_0 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=0
)
bc_u1_1 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=0
)
bc_v1_0 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=1
)
bc_v1_1 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=1
)
bc_u2_0 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=2
)
bc_u2_1 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=2
)
bc_v2_0 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=3
)
bc_v2_1 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=3
)
# (e^c=...)
# 初始条件


def boundary_l(x, on_boundary):
    return on_boundary and np.isclose(x[0], -1)


i = 1j
a = 2
sigma = 1
c_1 = 1+i
c_2 = -2-2*i
c_3 = 2+2*i
c_4 = -3+5*i
lambda1 = i*sigma
lambda11 = np.conj(lambda1)
v = i*np.sqrt(a**2-sigma**2)



# q_111=q1` q_1=q1 q_11=q1*


def init_cond_u1(x):
    q_1 = a*np.exp(i*2*a**2*x[:,1:2])
    q_2 = 0
    theta = i*x[:, 0:1]*v+2*i*x[:, 1:2]*v*np.abs(sigma)


    phi_1 = np.exp(i*a**2*x[:, 1:2])/(2*v)*(c_1*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                 v*(np.exp(-theta)-np.exp(theta)))+c_2*a*(np.exp(theta)-np.exp(-theta)))
    phi_2 = np.exp(i*a**2*x[:, 1:2])/(2*v)*(c_3*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                 v*(np.exp(-theta)-np.exp(theta)))+c_4*a*(np.exp(theta)-np.exp(-theta)))
    phi_3 = np.exp(-i*a**2*x[:, 1:2])/(2*v)*(c_2*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                  v*(np.exp(-theta)-np.exp(theta)))+c_1*a*(np.exp(theta)-np.exp(-theta)))
    phi_4 = np.exp(-i*a**2*x[:, 1:2])/(2*v)*(c_4*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                  v*(np.exp(-theta)-np.exp(theta)))+c_3*a*(np.exp(theta)-np.exp(-theta)))

    x_1 = phi_1*phi_3+phi_2*phi_4
    x_11 = np.conj(x_1)
    x_2 = np.abs(phi_1)**2+np.abs(phi_2)**2
    x_22 = np.conj(x_2)
    x_3 = np.abs(phi_3)**2+np.abs(phi_4)**2
    x_33 = np.conj(x_3)
    x_4 = phi_2*phi_3-phi_1*phi_4
    x_44 = np.conj(x_4)

    q_111 = q_1-2*i*(lambda1-lambda11)*x_2*x_11+x_1*x_33 / \
        (2*np.abs(x_1)**2+np.abs(x_2)**2+np.abs(x_3)**2-2*np.abs(x_4)**2)
    q_222 = q_2+2*i*(lambda1-lambda11)*x_4*x_33+x_2*x_44 / \
        (2*np.abs(x_1)**2+np.abs(x_2)**2+np.abs(x_3)**2-2*np.abs(x_4)**2)

    result = q_111.real

    # result=( (1/phi1)*(np.exp(-2*k1*k2*x[:,1:2] + k1*x[:,0:1] + nf1) * np.cos((-k1**2+k2**2)*x[:,1:2] - k2*x[:,0:1] - nf2))
    #       + (1/phi2)*(np.exp(-2*l1*l2*x[:,1:2] + l1*x[:,0:1] + ng1) * np.cos((-l1**2+l2**2)*x[:,1:2] - l2*x[:,0:1] - ng2))
    #      )
    return result


def init_cond_u2(x):
    q_1 = a*np.exp(i*2*a**2*x[:,1:2])
    q_2 = 0
    theta = i*x[:, 0:1]*v+2*i*x[:, 1:2]*v*np.abs(sigma)


    phi_1 = np.exp(i*a**2*x[:, 1:2])/(2*v)*(c_1*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                 v*(np.exp(-theta)-np.exp(theta)))+c_2*a*(np.exp(theta)-np.exp(-theta)))
    phi_2 = np.exp(i*a**2*x[:, 1:2])/(2*v)*(c_3*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                 v*(np.exp(-theta)-np.exp(theta)))+c_4*a*(np.exp(theta)-np.exp(-theta)))
    phi_3 = np.exp(-i*a**2*x[:, 1:2])/(2*v)*(c_2*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                  v*(np.exp(-theta)-np.exp(theta)))+c_1*a*(np.exp(theta)-np.exp(-theta)))
    phi_4 = np.exp(-i*a**2*x[:, 1:2])/(2*v)*(c_4*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                  v*(np.exp(-theta)-np.exp(theta)))+c_3*a*(np.exp(theta)-np.exp(-theta)))

    x_1 = phi_1*phi_3+phi_2*phi_4
    x_11 = np.conj(x_1)
    x_2 = np.abs(phi_1)**2+np.abs(phi_2)**2
    x_22 = np.conj(x_2)
    x_3 = np.abs(phi_3)**2+np.abs(phi_4)**2
    x_33 = np.conj(x_3)
    x_4 = phi_2*phi_3-phi_1*phi_4
    x_44 = np.conj(x_4)

    q_111 = q_1-2*i*(lambda1-lambda11)*x_2*x_11+x_1*x_33 / \
        (2*np.abs(x_1)**2+np.abs(x_2)**2+np.abs(x_3)**2-2*np.abs(x_4)**2)
    q_222 = q_2+2*i*(lambda1-lambda11)*x_4*x_33+x_2*x_44 / \
        (2*np.abs(x_1)**2+np.abs(x_2)**2+np.abs(x_3)**2-2*np.abs(x_4)**2)

    result = q_222.real

    # result=( -(1/phi1)*(np.exp(-2*k1*k2*x[:,1:2] + k1*x[:,0:1] + nf1) * np.sin((-k1**2+k2**2)*x[:,1:2] - k2*x[:,0:1] - nf2))
    #       + (1/phi2)*(np.exp(-2*l1*l2*x[:,1:2] + l1*x[:,0:1] + ng1) * np.sin((-l1**2+l2**2)*x[:,1:2] - l2*x[:,0:1] - ng2))
    #     )
    return result


def init_cond_v1(x):
    q_1 = a*np.exp(i*2*a**2*x[:,1:2])
    q_2 = 0
    theta = i*x[:, 0:1]*v+2*i*x[:, 1:2]*v*np.abs(sigma)


    phi_1 = np.exp(i*a**2*x[:, 1:2])/(2*v)*(c_1*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                 v*(np.exp(-theta)-np.exp(theta)))+c_2*a*(np.exp(theta)-np.exp(-theta)))
    phi_2 = np.exp(i*a**2*x[:, 1:2])/(2*v)*(c_3*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                 v*(np.exp(-theta)-np.exp(theta)))+c_4*a*(np.exp(theta)-np.exp(-theta)))
    phi_3 = np.exp(-i*a**2*x[:, 1:2])/(2*v)*(c_2*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                  v*(np.exp(-theta)-np.exp(theta)))+c_1*a*(np.exp(theta)-np.exp(-theta)))
    phi_4 = np.exp(-i*a**2*x[:, 1:2])/(2*v)*(c_4*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                  v*(np.exp(-theta)-np.exp(theta)))+c_3*a*(np.exp(theta)-np.exp(-theta)))

    x_1 = phi_1*phi_3+phi_2*phi_4
    x_11 = np.conj(x_1)
    x_2 = np.abs(phi_1)**2+np.abs(phi_2)**2
    x_22 = np.conj(x_2)
    x_3 = np.abs(phi_3)**2+np.abs(phi_4)**2
    x_33 = np.conj(x_3)
    x_4 = phi_2*phi_3-phi_1*phi_4
    x_44 = np.conj(x_4)

    q_111 = q_1-2*i*(lambda1-lambda11)*x_2*x_11+x_1*x_33 / \
        (2*np.abs(x_1)**2+np.abs(x_2)**2+np.abs(x_3)**2-2*np.abs(x_4)**2)
    q_222 = q_2+2*i*(lambda1-lambda11)*x_4*x_33+x_2*x_44 / \
        (2*np.abs(x_1)**2+np.abs(x_2)**2+np.abs(x_3)**2-2*np.abs(x_4)**2)

    result = -q_111.imag
    # result=-( (1/phi1)*(np.exp(-2*k1*k2*x[:,1:2] + k1*x[:,0:1] + nf1) * np.sin((-k1**2+k2**2)*x[:,1:2] - k2*x[:,0:1] - nf2))
    #        + (1/phi2)*(np.exp(-2*l1*l2*x[:,1:2] + l1*x[:,0:1] + ng1) * np.sin((-l1**2+l2**2)*x[:,1:2] - l2*x[:,0:1] - ng2))
    #       )
    return result


def init_cond_v2(x):
    q_1 = a*np.exp(i*2*a**2*x[:,1:2])
    q_2 = 0
    theta = i*x[:, 0:1]*v+2*i*x[:, 1:2]*v*np.abs(sigma)


    phi_1 = np.exp(i*a**2*x[:, 1:2])/(2*v)*(c_1*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                 v*(np.exp(-theta)-np.exp(theta)))+c_2*a*(np.exp(theta)-np.exp(-theta)))
    phi_2 = np.exp(i*a**2*x[:, 1:2])/(2*v)*(c_3*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                 v*(np.exp(-theta)-np.exp(theta)))+c_4*a*(np.exp(theta)-np.exp(-theta)))
    phi_3 = np.exp(-i*a**2*x[:, 1:2])/(2*v)*(c_2*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                  v*(np.exp(-theta)-np.exp(theta)))+c_1*a*(np.exp(theta)-np.exp(-theta)))
    phi_4 = np.exp(-i*a**2*x[:, 1:2])/(2*v)*(c_4*(sigma*(np.exp(theta)-np.exp(-theta)) +
                                                  v*(np.exp(-theta)-np.exp(theta)))+c_3*a*(np.exp(theta)-np.exp(-theta)))

    x_1 = phi_1*phi_3+phi_2*phi_4
    x_11 = np.conj(x_1)
    x_2 = np.abs(phi_1)**2+np.abs(phi_2)**2
    x_22 = np.conj(x_2)
    x_3 = np.abs(phi_3)**2+np.abs(phi_4)**2
    x_33 = np.conj(x_3)
    x_4 = phi_2*phi_3-phi_1*phi_4
    x_44 = np.conj(x_4)

    q_111 = q_1-2*i*(lambda1-lambda11)*x_2*x_11+x_1*x_33 / \
        (2*np.abs(x_1)**2+np.abs(x_2)**2+np.abs(x_3)**2-2*np.abs(x_4)**2)
    q_222 = q_2+2*i*(lambda1-lambda11)*x_4*x_33+x_2*x_44 / \
        (2*np.abs(x_1)**2+np.abs(x_2)**2+np.abs(x_3)**2-2*np.abs(x_4)**2)

    result = -q_222.imag

    # result= ( -(1/phi1)*(np.exp(-2*k1*k2*x[:,1:2] + k1*x[:,0:1] + nf1) * np.cos((-k1**2+k2**2)*x[:,1:2] - k2*x[:,0:1] - nf2))
    #        + (1/phi2)*(np.exp(-2*l1*l2*x[:,1:2] + l1*x[:,0:1] + ng1) * np.cos((-l1**2+l2**2)*x[:,1:2] - l2*x[:,0:1] - ng2))
    #       )
    return result


ic_u1 = dde.IC(geomtime, init_cond_u1, lambda _,
               on_initial: on_initial, component=0)
ic_v1 = dde.IC(geomtime, init_cond_v1, lambda _,
               on_initial: on_initial, component=1)
ic_u2 = dde.IC(geomtime, init_cond_u2, lambda _,
               on_initial: on_initial, component=2)
ic_v2 = dde.IC(geomtime, init_cond_v2, lambda _,
               on_initial: on_initial, component=3)

data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc_u1_0, bc_u1_1, bc_v1_0, bc_v1_1, bc_u2_0, bc_u2_1,
        bc_v2_0, bc_v2_1, ic_u1, ic_v1, ic_u2, ic_v2],
    num_domain=10000,
    num_boundary=200,
    num_initial=200,
    train_distribution="pseudo",
)

allow_unused = True
# 网络架构
net = dde.maps.FNN([2] + [100] * 6 + [4], "silu", "Glorot normal")

model = dde.Model(data, net)


Using backend: tensorflow.compat.v1



Instructions for updating:
non-resource variables are not supported in the long term
Enable just-in-time compilation with XLA.


C:\tools\Anaconda3\python.exe
C:\tools\Anaconda3\python.exe


In [2]:
model.compile("adam", lr=1e-3, loss="MSE")
model.train(epochs=40000, display_every=1000)


Compiling model...
Building feed-forward neural network...
'build' took 0.084218 s



C:\Users\npofsi\AppData\Roaming\Python\Python39\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:103: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(
C:\Users\npofsi\AppData\Roaming\Python\Python39\site-packages\keras\legacy_tf_layers\core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
C:\Users\npofsi\AppData\Local\Temp\ipykernel_26168\3146289385.py:186: RuntimeWarning: overflow encountered in multiply
  q_111 = q_1-2*i*(lambda1-lambda11)*x_2*x_11+x_1*x_33 / \
C:\Users\npofsi\AppData\Local\Temp\ipykernel_26168\3146289385.py:187: RuntimeWarning: overflow encountered in square
  (2*np.abs(x_1)**2+np.abs(x_2)**2+np.abs(x_3)**2-2*np.abs(x_4)**2)
C:\Users\npofsi\AppData\Local\Temp\ipykernel_26168\3146289385.py:187: RuntimeWarning: invalid value encountered in subtrac

'compile' took 5.920149 s

Initializing variables...
Training model...

Step      Train loss                                                                                                                                                          Test loss                                                                                                                                                           Test metric
0         [4.19e-04, 1.79e-05, 7.20e-06, 3.02e-04, 3.50e-02, 2.04e-03, 1.15e-04, 1.48e-03, 7.31e-03, 5.05e-06, 1.70e-02, 1.41e-03, nan, nan, nan, nan]                        [4.19e-04, 1.79e-05, 7.20e-06, 3.02e-04, 3.50e-02, 2.04e-03, 1.15e-04, 1.48e-03, 7.31e-03, 5.05e-06, 1.70e-02, 1.41e-03, nan, nan, nan, nan]                        []  

Best model at step 0:
  train loss: inf
  test loss: inf
  test metric: 

'train' took 20.205573 s



(<deepxde.model.LossHistory at 0x17ee51ffe20>,
 <deepxde.model.TrainState at 0x17ee51ffa30>)

In [ ]:
dde.optimizers.config.set_LBFGS_options(
    maxcor=50,
    ftol=1.0 * np.finfo(float).eps,
    gtol=1e-09,
    maxiter=10000,
    maxfun=5000,
    maxls=50,
)
model.compile("L-BFGS")
# model.train()

losshistory, train_state = model.train()
dde.saveplot(losshistory, train_state, issave=True, isplot=True)


: 

In [ ]:
# 做预测
prediction = model.predict(X_star, operator=None)

u1 = griddata(X_star, prediction[:, 0], (X, T), method="cubic")
v1 = griddata(X_star, prediction[:, 1], (X, T), method="cubic")
u2 = griddata(X_star, prediction[:, 2], (X, T), method="cubic")
v2 = griddata(X_star, prediction[:, 3], (X, T), method="cubic")


h1 = np.sqrt(u1 ** 2 + v1 ** 2)
h2 = np.sqrt(u2**2+v2**2)


# 绘制预测
fig, ax = plt.subplots(3)

ax[0].set_title("Results1")
ax[0].set_ylabel("Real part1")
ax[0].imshow(
    u1.T,
    interpolation="nearest",  # 插值
    cmap="viridis",  # 色彩图
    extent=[t_lower, t_upper, x_lower, x_upper],  # 极值
    origin="lower",
    aspect="auto",
)
ax[1].set_ylabel("Imaginary part1")
ax[1].imshow(
    v1.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
ax[2].set_ylabel("Amplitude1")
ax[2].imshow(
    h1.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)

fig, bx = plt.subplots(3)
bx[0].set_title("Results2")
bx[0].set_ylabel("Real part2")
bx[0].imshow(
    u2.T,
    interpolation="nearest",  # 插值
    cmap="viridis",  # 色彩图
    extent=[t_lower, t_upper, x_lower, x_upper],  # 极值
    origin="lower",
    aspect="auto",
)
bx[1].set_ylabel("Imaginary part2")
bx[1].imshow(
    v2.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
bx[2].set_ylabel("Amplitude1")
bx[2].imshow(
    h2.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
plt.show()


: 

In [ ]:
ax = plt.axes(projection='3d')


b = prediction[:, 0]**2+prediction[:, 1]**2
c = min(b)
d = b/c
d = d.astype(int)
print(d)
# Data for a three-dimensional line
ax.scatter3D(X_star[:, 0], X_star[:, 1], b, c=d[:], cmap='plasma')


: 

In [ ]:
bx = plt.axes(projection='3d')
b = prediction[:, 2]**2+prediction[:, 3]**2
c = min(b)
d = b/c
d = d.astype(int)
print(d)
bx.scatter3D(X_star[:, 0], X_star[:, 1], b, c=d[:]/10)

print(X.shape)
print(T.shape)
print(X_star.shape)
print(prediction.shape)


: 